In [1]:
import numpy as np
import numpy.random as npr
import cvxpy as cp

### Define Functions

In [7]:
def greedy_lower_bound(K, w, r): # Greedy Lower bound
    m,n = K.shape
    K_bar = 1-K
    v = np.zeros(n)
    for i in range(n):
        v[i] = np.ones(m) @ K_bar[:, i] * w[i]

    v_srtd = np.flip(np.argsort(v))
    v[v_srtd]

    # v_srtd is the arguments of the "most important" features to pick

    x = np.zeros(n)
    for i in range(n):
        x_temp = np.copy(x)
        x_temp[v_srtd[i]] = 1
        if np.count_nonzero(K @ x_temp) <= m*(1-r):
            x = x_temp

    # COUNT IS LOWER BOUND
    count = x @ w
    return count, x

In [20]:
def solve_mip_upper_bound(K, w, r): # LP upper bound
    m,n = K.shape
    K_bar = 1 - K
    x = cp.Variable(n, boolean = True)
    obj = cp.Maximize(w @ x)
    # cons = [cp.norm(K @ x, p = 0) <= m * (1-r)]
    cons = [r * m * cp.sum(x) - np.ones(m) @ K_bar @ x <= 0]
    # cons += [x <= 1, 0 <= x]

    prob = cp.Problem(obj, cons)
    prob.solve()
    return prob.value, x.value

In [21]:
def solve_mip_lower_bound(K, w, r): # LP upper bound
    m,n = K.shape
    x = cp.Variable(n, boolean = True)
    obj = cp.Maximize(w @ x)
    # cons = [cp.norm(K @ x, p = 0) <= m * (1-r)]
    cons = [np.ones(m) @ K @ x <= m * (1-r)]
    # cons += [x <= 1, 0 <= x]

    prob = cp.Problem(obj, cons)
    prob.solve()
    return prob.value, x.value

In [36]:
def sdp_relaxation(K,w,r): # poor sdp relaxation
    m,n = K.shape
    K_bar = 1 - K
    X = cp.Variable((n+m+1,n+m+1), PSD = True)

    obj = cp.Maximize(w @ X[n+m,:n])
    cons = []
    cons += [cp.sum(X[n+m,n:n+m]) >= r*m]

    for i in range(n):
        cons += [X[i,i] == X[n+m,i]]
    for j in range(n, n+m):
        cons += [X[j,j] == X[n+m,j]]
    cons += [X[m+n,m+n] == 1]
    for i in range(n):
        for j in range(m):
            if K[j,i] == 1:
                cons += [X[i,n+j] == 0]
    # cons += [X[n+m,:n] <= 1, 0 <= X[n+m,:n]]
    cons += [X[n+m,:n+m] <= 1, 0 <= X[n+m,:n+m]]
    cons += [r * m * cp.sum(X[n+m,:n]) - np.ones(m) @ K_bar @ X[n+m,:n] <= 0]


    prob = cp.Problem(obj, cons)

    prob.solve()
    return prob.value, X.value

In [37]:
def true_const_sat(K, r, x): # is the actual constraint satisfied
    m,_ = K.shape
    return np.count_nonzero(K @ x) <= m*(1-r)

## Make random data matrix

In [64]:
npr.seed(5)

m = 20
n = 15
k = 0

D = npr.randint(2, size = (m,n))
K = np.abs(D - D[k,:])
K_bar = 1-K

# information weighting vector
w = npr.randint(0, n, size = n)
r = 0.6

In [65]:
p_val_low, x_val_low = greedy_lower_bound(K,w,r)
p_val_mip_high, x_val_mip_high = solve_mip_upper_bound(K,w,r)
p_val_mip_low, x_val_mip_low = solve_mip_lower_bound(K,w,r)

bound_tight = true_const_sat(K, r, x_val_mip_high)

p_val_high_sdp, x_val_high_sdp = sdp_relaxation(K,w,r)

In [66]:
print("Upper bound MIP = ", p_val_mip_high)
print("Lower bound MIP = ", p_val_mip_low)
print("Lower_bound = ", p_val_low)
print("upper bound sdp = ", p_val_high_sdp)
print("upper bound LP tight = ", true_const_sat(K,r, x_val_mip_high))


Upper bound MIP =  39.0
Lower bound MIP =  7.0
Lower_bound =  7.0
upper bound sdp =  19.223049401785378
upper bound LP tight =  False
